## Высокоуровневый дизайн


## Что такое высокоуровневый дизайн?
Под высокоуровневым дизайном мы будем понимать такой дизайн, который минимально учитывает основные сценарии взаимодействия с системой, но при этом не отражает всю сложность взаимодействия между большим количеством реальных компонентов.

> Можно исходить из того, что реализации высокоуровневого дизайна было бы условно достаточно для демонстрации MVP вашего продукта на ноутбуке одного из фаундеров.

### Общий подход к дизайну
В голове можно всегда держать общий шаблон (pattern), подходящий почти для всего:
1.  **Front-end**: Часть системы взаимодействует непосредственно с юзером.
2.  **Back-end**: Часть системы работает скрыто от пользователя "под капотом".
3.  **Storage**: Часть системы заточена на долгосрочное хранение данных (базы данных, файловые системы).

### Адаптация под специфику задачи
*   **Онлайн-кинотеатр**: Важна работа с "тяжелыми" файлами -> добавляем специализированное хранилище (Object Storage).
*   **Такси**: Взаимодействие разных типов пользователей (пассажиры и водители) -> разные приложения/интерфейсы.
*   **Миллионы пользователей**: Сразу отражаем необходимость **балансировки нагрузки (Load Balancer)**.

---


### Сокращатель ссылок

**Зачем нужен?** Превращать длинные ссылки в короткие.

**Первоочередные особенности:**
*   Простой функционал: прочитать (redirect) или создать (create).
*   Можно выделить два модуля:
    1.  **API**: Отвечает на запросы пользователей.
    2.  **Generator**: Генерирует уникальные короткие ключи (Key Generation Service).
*   **Масштабирование**: Ключи не должны повторяться между серверами. Сервера должны синхронизироваться или использовать заранее сгенерированные диапазоны ключей.


### Хостинг текстов (Pastebin)

**Зачем нужен?** Делиться кусками кода.

**Первоочередные особенности:**
*   Тексты могут быть любого размера (от 1KB до 10MB).
*   **Разделение хранения**: 
    *   Метаданные (автор, дата, настройки) -> **База Данных**.
    *   Сам текст (контент) -> **Object Storage** (S3) или Block Storage.
*   Сервер общается с двумя хранилищами: одной для метаданных, другой для больших текстов.


### Автодополнение

**Зачем нужен?** Предлагать варианты при вводе (Typeahead).

**Первоочередные особенности:**
*   Сервис хранит популярные запросы.
*   Требуется специальная структура данных для быстрого поиска по префиксу (Trie / Prefix Tree).
*   **Архитектура**:
    *   Сервер логирует новые запросы (сбор статистики).
    *   Периодически строится/обновляется индекс для поиска.
    *   При чтении (поиске) идем в этот подготовленный индекс.


### Облачный диск (Google Drive)

**Зачем нужен?** Хранение и синхронизация файлов.

**Первоочередные особенности:**
*   Файлы любого размера.
*   Разделение на **Metadata DB** (структура папок, права) и **Block Storage** (сами файлы).
*   **Клиент**: Должен уметь разбивать файлы на чанки (chunks), дедуплицировать, загружать/скачивать.
*   **Синхронизация**: Компонент уведомлений, чтобы другие устройства узнали об изменениях.


### Фотоаппы (Instagram)

**Зачем нужен?** Делиться фото.

**Первоочередные особенности:**
*   Два основных сценария: **Загрузка** (Write) и **Просмотр ленты** (Read).
*   Фотографии занимают много места -> Object Storage.
*   Метаданные (лайки, комменты) -> SQL/NoSQL DB.
*   Можно разделить на микро-сервисы: сервис загрузки и сервис чтения ленты.


### Телеграм

**Зачем нужен?** Мессенджер.

**Первоочередные особенности:**
*   **Real-time**: Необходимо быстро доставлять сообщения (Push Notifications, WebSocket).
*   **История**: Хранение переписки (синхронизация между устройствами).
*   **Статус**: Сервис присутствия (Online/Offline/Last seen).
*   Сервера должны знать, к какому физическому серверу подключен конкретный пользователь сейчас (Session Service).


### Твиттер

**Зачем нужен?** Публикация коротких сообщений.

**Первоочередные особенности:**
*   Огромное количество чтений (Fan-out).
*   **Балансировка нагрузки** критически важна.
*   Разделение "тяжелого" контента (медиа) и текста твитов.
*   Алгоритмическая лента -> сервис рекомендаций.


### Нетфликс

**Зачем нужен?** Стриминг видео.

**Первоочередные особенности:**
*   **CDN (Content Delivery Network)**: Критически важен для доставки тяжелого контента по всему миру.
*   **Transcoding**: Подготовка видео в разных разрешениях/форматах для разных устройств заранее.
*   Приложение общается с API (каталог, поиск) и отдельно тянет видео-чанки с CDN.


### Сервис такси (Uber)

**Зачем нужен?** Поездки.

**Первоочередные особенности:**
*   Два разных приложения: **Driver App** и **Rider App**.
*   **Geo-Service**: Работа с гео-индексами, поиск ближайших водителей.
*   **Dispatch Service**: Матчинг водителя и пассажира.
*   Хранение активного состояния (Trip) и истории (Archive).
